<a href="https://colab.research.google.com/github/stayup24h/Hangul-to-Unicode-Obfuscation-Project/blob/main/model_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# initial
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
#모델 설계

def model(input_shape, num_classes):
    model = Sequential()

    # CNN 레이어
    # Conv1_1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2))) # (None, 128, 128, 64)

    # Conv2_1
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2))) # (None, 64, 64, 128)

    # Conv3_1, Conv3_2
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 1))) # (None, 32, 64, 256)

    # Conv4_1
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.BatchNormalization())

    # Conv5_1
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(MaxPooling2D((2, 1))) # (None, 16, 64, 512)

    # Conv6_1
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 1))) # (None, 8, 64, 512)

    # RNN 레이어를 위한 Reshape
    new_width_for_rnn = 64 # CNN 출력의 width
    features_per_timestep = 1 * 512 # CNN 출력의 height * channels
    model.add(Reshape(target_shape=(new_width_for_rnn, features_per_timestep))) # (None, 64, 512)

    # RNN 레이어 (양방향 LSTM)
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))

    # 출력 레이어
    model.add(Dense(num_classes, activation='softmax'))

    return model

In [ ]:
# 모델 체크포인트 관리 클래스
class CheckpointManager:
    def __init__(self, max_recent=3, extra_interval=50):
        self.max_recent = max_recent
        self.extra_interval = extra_interval
        self.recent_epochs = []

    def load(self, model, epoch, checkpoint_path):
        path = checkpoint_path.format(epoch=epoch)
        if os.path.exists(path):
            model.load_weights(path)
            print(f"Loaded weights from {path}")
            return True
        else:
            print(f"No checkpoint found at {path}")
            return False

    def save(self, model, epoch):
        # Always save at extra_interval epochs
        if epoch % self.extra_interval == 0 and epoch not in self.recent_epochs:
            path = self.checkpoint_path_template.format(epoch=epoch)
            model.save_weights(path)
            print(f"Extra checkpoint saved for epoch {epoch} at {path}")

        # Save recent checkpoints
        self.recent_epochs.append(epoch)
        if len(self.recent_epochs) > self.max_recent:
            # Remove oldest checkpoint from recent
            old_epoch = self.recent_epochs.pop(0)
            old_path = self.checkpoint_path_template.format(epoch=old_epoch)
            if os.path.exists(old_path):
                os.remove(old_path)
                print(f"Removed old checkpoint at {old_path}")

        path = self.checkpoint_path_template.format(epoch=epoch)
        model.save_weights(path)
        print(f"Checkpoint saved for epoch {epoch} at {path}")

In [ ]:
# 데이터 로드 및 전처리 함수

# 데이터셋 경로 설정 (실제 경로에 맞게 수정 필요)
image_dir = "./data/images"
json_dir = "./data/labels"

# 모델 입력 형태 및 클래스 수 정의 (실제 데이터에 맞게 조정 필요)
# CRNN 모델의 입력 형태: (높이, 너비, 채널)
input_shape = (21, 256, 1)  
num_classes = 87 # 라벨로 사용되는 class 수

def load_and_preprocess_image(image_path, target_size=(64, 64)):
    img = load_img(image_path, color_mode='grayscale', target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # 정규화
    return img_array

def load_and_preprocess_json_label(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        label_data = json.load(f)
    # JSON 파일에서 실제 레이블을 추출하는 로직 추가 필요
    # 예시: 'label' 키에 해당하는 값을 반환
    # 이 부분은 실제 JSON 파일 구조에 따라 달라집니다.
    if 'label' in label_data:
        return label_data['label']
    else:
        raise ValueError(f"JSON 파일에 'label' 키가 없습니다: {json_path}")

# 데이터 제너레이터 (훈련 데이터 로드를 위한)
def data_generator(image_filenames, batch_size, input_shape, num_classes):
    num_samples = len(image_filenames)
    while True:
        # 배치 단위로 데이터 셔플 및 로드
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i + batch_size]
            batch_images = []
            batch_labels = []

            for j in batch_indices:
                image_filename = image_filenames[j]
                image_path = os.path.join(image_dir, image_filename)
                json_filename = image_filename.replace('.png', '.json').replace('.jpg', '.json') # 이미지 확장자에 따라 변경
                json_path = os.path.join(json_dir, json_filename)

                # 이미지 로드 및 전처리
                image = load_and_preprocess_image(image_path, target_size=input_shape[:2])
                batch_images.append(image)

                # JSON 레이블 로드 및 전처리
                label_value = load_and_preprocess_json_label(json_path)
                # 레이블을 원-핫 인코딩 또는 적절한 형태로 변환하는 로직 추가 필요
                # 이 부분은 num_classes와 레이블의 종류에 따라 달라집니다.
                # 예시: 정수 레이블을 원-핫 인코딩
                one_hot_label = tf.keras.utils.to_categorical(label_value, num_classes=num_classes)
                batch_labels.append(one_hot_label)

            yield np.array(batch_images), np.array(batch_labels)


In [ ]:
# 모델 인스턴스 생성 및 컴파일
model = model(input_shape=input_shape, num_classes=num_classes)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


checkpoint_manager = CheckpointManager(
    max_recent=5,  # 최근 5개의 체크포인트 유지
    extra_interval=10 # 10 에포크마다 추가 체크포인트 저장
)

# 최신 체크포인트 로드 시도
start_epoch = 0
# 모든 이미지 파일 이름 가져오기
all_image_filenames = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# 가장 최근에 저장된 에포크 찾기
if checkpoint_manager.recent_epochs:
    latest_epoch = max(checkpoint_manager.recent_epochs)
    if checkpoint_manager.load(model, latest_epoch):
        start_epoch = latest_epoch + 1
        print(f"restart train {start_epoch} 에포크부터 재개합니다.")
else:
    print("저장된 체크포인트가 없습니다. 모델을 처음부터 훈련합니다.")

# 훈련 파라미터 설정
batch_size = 32
target_accuracy = 0.98  # 목표 정답률

# 데이터 제너레이터 생성
train_generator = data_generator(all_image_filenames, batch_size, input_shape, num_classes)

# 훈련 루프
epoch = start_epoch
while True:
    print(f"\n에포크 {epoch} 시작...")

    # 한 에포크 동안 훈련
    steps_per_epoch = len(all_image_filenames) // batch_size
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=1,  # 한 번에 한 에포크씩 훈련
        verbose=1
    )

    # 현재 에포크의 정답률 확인
    current_accuracy = history.history['accuracy'][0]
    print(f"에포크 {epoch} 정답률: {current_accuracy:.4f}")

    # 체크포인트 저장
    checkpoint_manager.save(model, epoch)

    # 목표 정답률 달성 여부 확인
    if current_accuracy >= target_accuracy:
        print(f"목표 정답률 {target_accuracy:.4f}에 도달했습니다. 훈련을 중단합니다.")
        break

    epoch += 1
